In [ ]:
!pip install mwclient
!pip install transformers

In [19]:
import mwclient
import time
from transformers import pipeline
from statistics import mean
import pandas as pd
from datetime import datetime

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
site = mwclient.Site("en.wikipedia.org")
page = site.pages["Bitcoin"]
revs = list(page.revisions())
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
  sent = sentiment_pipeline([text[:250]])[0]
  score=sent["score"]
  if sent["label"] == "NEGATIVE":
    score *= -1
  return score

In [21]:
edits = {}

for rev in revs:
  date = time.strftime("%Y-%m-%d", rev["timestamp"])

  if date not in edits:
    edits[date] = dict(sentiments=list(), edit_count=0)

  try:
    edits[date]["edit_count"] += 1
    comment = rev["comment"]
    edits[date]["sentiments"].append(find_sentiment(comment))
  except Exception as e:
    continue

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
for key in edits:
  if len(edits[key]["sentiments"]) > 0:
    edits[key]["sentiment"] = mean(edits[key]["sentiments"])
    edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
  else:
    edits[key]["sentiment"] = 0
    edits[key]["neg_sentiment"] = 0

  del edits[key]["sentiments"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
edits_df = pd.DataFrame.from_dict(edits, orient="index")
edits_df.index = pd.to_datetime(edits_df.index)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
dates = pd.date_range(start="2009-03-08", end=datetime.today())

edits_df =edits_df.reindex(dates, fill_value=0)
rolling_edits = edits_df.rolling(30).mean()
rolling_edits = rolling_edits.dropna()
rolling_edits.to_csv("wikipedia_edits.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
